In [1]:
import pandas as pd
file_path = "data/0613.csv"
df = pd.read_csv(file_path)
df.head()

,元智大學 1132(CM254C),Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8
0,下載日期:2025/6/13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,點名日期,課號,課名,節次,學號,姓名,經度,緯度,簽到時間
3,20250307,1132-CM254C,機器學習應用,第 02 節,1101444,徐英豈,24.968098,121.26738,45723.37974
4,20250307,1132-CM254C,機器學習應用,第 02 節,1102455,江毓婷,24.970192,121.263435,45723.39526


In [2]:
# 重新設置標題行，並刪除不必要的行
df_cleaned = df.iloc[2:].reset_index(drop=True)
df_cleaned.columns = df_cleaned.iloc[0]  # 設置新標題
df_cleaned = df_cleaned[1:].reset_index(drop=True)  # 刪除舊標題行

# 重命名關鍵欄位
df_cleaned = df_cleaned.rename(columns={"姓名": "Name", "經度": "Latitude", "緯度": "Longitude", "簽到時間": "Check-in Time"})

# 轉換數據類型
df_cleaned["Latitude"] = pd.to_numeric(df_cleaned["Latitude"], errors="coerce")
df_cleaned["Longitude"] = pd.to_numeric(df_cleaned["Longitude"], errors="coerce")

# 轉換簽到時間（Excel 時間格式轉換）
df_cleaned["Check-in Time"] = pd.to_numeric(df_cleaned["Check-in Time"], errors="coerce")
df_cleaned["Check-in Time"] = pd.to_datetime(df_cleaned["Check-in Time"], unit="D", origin="1899-12-30")

df_cleaned

,點名日期,課號,課名,節次,學號,Name,Latitude,Longitude,Check-in Time
0,20250307,1132-CM254C,機器學習應用,第 02 節,1101444,徐英豈,24.968098,121.267380,2025-03-07 09:06:49.535999714
1,20250307,1132-CM254C,機器學習應用,第 02 節,1102455,江毓婷,24.970192,121.263435,2025-03-07 09:29:10.463999784
2,20250307,1132-CM254C,機器學習應用,第 02 節,1102514,王 杉,24.976917,121.255814,2025-03-07 09:18:27.648000311
3,20250307,1132-CM254C,機器學習應用,第 02 節,1102527,簡佳媛,25.021704,121.255318,2025-03-07 09:12:55.871999887
4,20250307,1132-CM254C,機器學習應用,第 02 節,1103002,林郁綺,24.970827,121.267715,2025-03-07 09:18:57.024000190
...,...,...,...,...,...,...,...,...,...
267,20250613,1132-CM254C,機器學習應用,第 02 節,1102514,王 杉,24.976784,121.255898,2025-06-13 09:12:21.312000129
268,20250613,1132-CM254C,機器學習應用,第 02 節,1112214,張婷婷,24.970716,121.267769,2025-06-13 09:26:37.536000190
269,20250613,1132-CM254C,機器學習應用,第 02 節,1112221,陳渝方,24.984583,121.278030,2025-06-13 09:15:40.032000138
270,20250613,1132-CM254C,機器學習應用,第 02 節,1112513,羅亦婷,24.970293,121.267410,2025-06-13 09:19:04.800000311


In [3]:
import folium

# 設定地圖中心為所有學生簽到位置的平均值
map_center = [df_cleaned["Latitude"].mean(), df_cleaned["Longitude"].mean()]
m = folium.Map(location=map_center, zoom_start=14)

locations = []
# 在地圖上添加標記
for _, row in df_cleaned.iterrows():
    popup_text = f"姓名: {row['Name']}<br>簽到時間: {row['Check-in Time']}"
    location = [row["Latitude"], row["Longitude"]]
    locations.append(location)
    folium.Marker(
        location=[row["Latitude"], row["Longitude"]],
        popup=folium.Popup(popup_text, max_width=300),
        icon=folium.Icon(color="blue", icon="info-sign")
    ).add_to(m)

# 自動調整地圖視窗範圍以包含所有標記
if locations:
    m.fit_bounds(locations)

# 保存地圖為 HTML 文件
html_file_path = "test.html"
m.save(html_file_path)

# 提供下載連結
html_file_path


'test.html'

In [4]:
from sklearn.cluster import KMeans
import folium
import matplotlib.pyplot as plt

# 擷取經緯度資料
coords = df_cleaned[["Latitude", "Longitude"]]

# 使用 KMeans 分群
kmeans = KMeans(n_clusters=3, random_state=0)
df_cleaned["Cluster"] = kmeans.fit_predict(coords)

# 建立 Folium 地圖
map_center = [coords["Latitude"].mean(), coords["Longitude"].mean()]
m_clustered = folium.Map(location=map_center, zoom_start=16)

# 設定每個群體的顏色
colors = ['red', 'green', 'blue']

# 加入標記到地圖
for _, row in df_cleaned.iterrows():
    folium.CircleMarker(
        location=[row["Latitude"], row["Longitude"]],
        radius=5,
        color=colors[row["Cluster"]],
        fill=True,
        fill_color=colors[row["Cluster"]],
        fill_opacity=0.7,
        popup=f"姓名: {row['Name']}<br>群體: {row['Cluster']}<br>簽到時間: {row['Check-in Time']}"
    ).add_to(m_clustered)

# 地圖調整至所有點的位置
m_clustered.fit_bounds(coords.values.tolist())

# 儲存為 HTML
html_clustered_path = "clustered_signins.html"
m_clustered.save(html_clustered_path)

html_clustered_path


'clustered_signins.html'

In [8]:
import folium
from shapely.geometry import Polygon

# 座標（已轉換為十進位格式）
coords = [
    (24.970306, 121.263250),
    (24.969944, 121.263333),
    (24.969972, 121.265611),
    (24.968972, 121.265944),
    (24.965083, 121.267139),
    (24.965083, 121.268278),
    (24.966583, 121.269583),
    (24.970972, 121.268778),
    (24.971194, 121.267999),
    (24.970999, 121.267111),
    (24.970583, 121.266000)
]

# 計算地圖中心
center_lat = sum(p[0] for p in coords) / len(coords)
center_lon = sum(p[1] for p in coords) / len(coords)

# 建立 Folium 地圖
m = folium.Map(location=[center_lat, center_lon], zoom_start=17)

# 繪製多邊形
folium.Polygon(
    locations=coords,
    color='blue',
    weight=2,
    fill=True,
    fill_color='blue',
    fill_opacity=0.4,
).add_to(m)

# 可選：標記每個點
for idx, (lat, lon) in enumerate(coords):
    folium.Marker(location=[lat, lon], popup=f'點 {idx+1}').add_to(m)

# 顯示地圖
m

In [9]:
from shapely.geometry import Point, Polygon 
# 計算地圖中心
center_lat = sum(p[0] for p in coords) / len(coords)
center_lon = sum(p[1] for p in coords) / len(coords)

# 判斷是否在校園範圍內
campus_polygon = Polygon(coords)
df_cleaned["InCampus"] = df_cleaned.apply(
    lambda row: campus_polygon.contains(Point(row["Latitude"], row["Longitude"])), axis=1
)

# 建立地圖
campus_map = folium.Map(location=[center_lat, center_lon], zoom_start=17)

# 畫出校園範圍
folium.Polygon(
    locations=coords,
    color='blue',
    weight=2,
    fill=True,
    fill_color='blue',
    fill_opacity=0.2,
).add_to(campus_map)

# 畫出標記
for _, row in df_cleaned.iterrows():
    color = "green" if row["InCampus"] else "red"
    folium.CircleMarker(
        location=[row["Latitude"], row["Longitude"]],
        radius=5,
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=0.7,
        popup=f"姓名: {row['Name']}<br>簽到時間: {row['Check-in Time']}"
    ).add_to(campus_map)

# 儲存地圖
campus_map_path = "campus_signins.html"
campus_map.save(campus_map_path)

campus_map_path

'campus_signins.html'